### 1.2 Filtered Boston housing and kernels

Downloaded csv in command line using:
`curl -o boston-filter.csv http://www0.cs.ucl.ac.uk/staff/M.Herbster/boston-filter/Boston-filtered.csv`


In [16]:
import numpy as np
import python_functions as py_func

# dataset_np_headers_dropped
ds = np.genfromtxt('boston-filter.csv', delimiter=',', skip_header=1)

#### Naive Regression
a. Using polynomial basis for k=1 only i.e. $y = b$.

b. The constant function effectively determines the bias ($y$-intercept) of the linear regression.
It is the lowest predicted value of the dependent variable, the median house price.

In [3]:
MSEs_train_part_a, MSEs_test_part_a = py_func.split_dataset_and_compute_20_MSEs_with_ones(ds)

c. For each of the 12 attributes, perform a linear regression using only the single attribute but incorporating a bias term so that the inputs are augmented with an additional 1 entry, (xi , 1), so that we learn a weight vector w
 ∈ R2.

(Does this mean they want all 12 weights or the average of the 12 weights or MSEs .. ?)

In [14]:
MSEs_train_part_c, MSEs_test_part_c = py_func.split_dataset_and_compute_20_MSEs_with_single_attr(ds)

means_train, means_test = [], []
for MSEs_train_part_c_per_attr, MSEs_test_part_c_per_attr in zip(MSEs_train_part_c, MSEs_test_part_c):
    means_train.append(np.mean(MSEs_train_part_c_per_attr))
    means_test.append(np.mean(MSEs_test_part_c_per_attr))

print(f'Means for each of the 12 attributes in train ds = \n{means_train}\n')
print(f'Means for each of the 12 attributes in test ds = \n{means_test}')

Means for each of the 12 attributes in train ds= 
[70.7041647837489, 72.03692002816041, 64.05057020382704, 80.34603836491502, 68.1197348390914, 43.675830130679536, 71.36935623649204, 78.04456954554622, 71.55957656203262, 65.32423821973977, 62.090915760503876, 37.47723015145918]

Means for each of the 12 attributes in test ds= 
[74.4874685503509, 76.79322964603105, 66.16293396472426, 85.56934661294534, 71.1069732261131, 43.808498207167375, 74.93678917645552, 81.79524781858404, 73.52533896219495, 67.23687830792827, 64.18149020991304, 40.87963802805747]


d. Perform linear regression using all of the data attributes at once.
Perform linear regression on the training set using this regressor, and incorporate a bias term as above.

Calculate the MSE on the training and test sets and note down the results.
You should find that this method outperforms any of the individual regressors.

In [5]:
MSEs_train_part_d, MSEs_test_part_d = py_func.split_dataset_and_compute_20_MSEs_with_all_12_attr(ds)

In [15]:
print(f'Mean MSE for train dataset, using all 12 attributes = {np.mean(MSEs_train_part_d)}')  # gives 65.39992873551633
print(f'Mean MSE for test dataset, using all 12 attributes = {np.mean(MSEs_test_part_d)}')  # gives 68.3736527258721

Mean MSE for train dataset, using all 12 attributes = 21.70373755394515
Mean MSE for test dataset, using all 12 attributes = 25.273765249937956


#### 1.3 Kernelised ridge regression

A Kernel function is given as an element-wise product ?:
$K_{i,j} = K(x_i, x_j)$
$l$ is the size of the training set.
$\gamma$ is the regularisation parameter.
$\sigma$ is a parameter of the Gaussian kernel.


In [64]:
def gaussian_kernel(X, sigma: float):
    num_of_rows_of_x = X.shape[0]
    kernel_values = np.empty((num_of_rows_of_x, num_of_rows_of_x))
    for i in range(num_of_rows_of_x):
        for j in range(num_of_rows_of_x):
            pairwise_difference = X[i] - X[j]
            sqrd_norm = np.square(np.linalg.norm(pairwise_difference))
            kernel_values[i][j] = np.exp(-1 * sqrd_norm / 2 * np.square(sigma))
    return kernel_values

In [65]:
g_dataset_30x, g_dataset_30y = py_func.generate_dataset_about_g(num_of_data_pairs=30)


In [53]:
# Create a vector of gamma values [2^-40, 2^-39,...,2^-26]
gammas = [2**pow for pow in list(range(-40, -25))]
# Create vector of sigma values [2^7, 2^7.5, . . . , 2^12.5, 2^13]
sigmas = []
for pow in list(range(7, 14)):
    sigmas.append(2**pow)
    sigmas.append(2**(pow+0.5))
sigmas = sigmas[:-1]

res = []
for sigma in sigmas:

    res.append(gaussian_kernel(g_dataset_30x, sigma))
res = np.array(res)
print(res.shape)

(13, 30)


In [66]:
res = gaussian_kernel(g_dataset_30x, sigmas[0])
res.shape

(30, 30)

In [73]:
def a_star(sigma, dataset_x, gamma, dataset_y):
    kernel_matrix = gaussian_kernel(dataset_x, sigma)
    l = dataset_x.shape[0]
    return (np.linalg.inv(kernel_matrix + gamma * l * np.identity(l))) @ dataset_y

In [76]:
astar = a_star(sigma=128, dataset_x=g_dataset_30x, gamma=2**-40, dataset_y=g_dataset_30y)
astar[0]

-0.38638126724692334

In [ ]:
def evaluation_of_regression(alpha_star, X_train, X_test_data_point):
    y_test = []
    for i in range(X_train.shape[0]):
        bla = alpha_star[i]
        print(bla.shape)

    return 0



In [63]:
from sklearn.model_selection import train_test_split
ds = np.genfromtxt('boston-filter.csv', delimiter=',', skip_header=1)
train_dataset, test_dataset = train_test_split(ds, test_size=1 / 5)
def get_x_train_y_train_x_test_y_test(m_train: int, train_ds, m_test: int, test_ds) -> tuple:
    X_train_all_attr = train_ds[:, 0: 12]
    ones_train = np.ones((m_train, 1))
    X_train = np.column_stack((ones_train, X_train_all_attr))
    y_train = train_ds[:, -1]
    X_test_all_attr = test_ds[:, 0: 12]
    ones_test = np.ones((m_test, 1))
    X_test = np.column_stack((ones_test, X_test_all_attr))
    y_test = test_ds[:, -1]
    return X_train, y_train, X_test, y_test

m_train, m_test = train_dataset.shape[0], test_dataset.shape[0]
X_train, y_train, X_test, y_test = get_x_train_y_train_x_test_y_test(m_train=m_train, train_ds=train_dataset,
                                                                     m_test=m_test, test_ds=test_dataset)

print(X_train.shape)
print(X_test.shape)

(404, 13)
(102, 13)


In [ ]:
gaussian_kernel_of_test_point(X_test, X_test_point, sigma):
    num_of_rows_of_x = X_test.shape[0]
    sqrd_norm = np.empty(num_of_rows_of_x)
    for i in range(num_of_rows_of_x):
        pairwise_difference = X_test[i] - X_test_point
        sqrd_norm[i] = np.square(np.linalg.norm(pairwise_difference))
    return np.exp(-1 * sqrd_norm / 2 * sigma ** 2)

In [78]:
import numpy as np

bla = np.ones(2)
bla

array([1., 1.])

In [81]:
np.sum(bla, axis=0)

2.0

In [95]:
import numpy as np
a = np.zeros((2,2))
a[0,0] = 1
a[0,1] = 2
a[1,0] = 3
a[1,1] = 4
b = np.zeros((2,2))
b[0,0] = 2
b[0,1] = 2
b[1,0] = 3
b[1,1] = 2
print(a)
print(b)
c = (a + b) / 2
print(c)

[[1. 2.]
 [3. 4.]]
[[2. 2.]
 [3. 2.]]
[[1.5 2. ]
 [3.  3. ]]


In [ ]:
mean_of_5folds=[[9702836.85669036, 13996061.59304154, 19224107.1127768524659123.90175939,
  29047858.03471764, 31935351.63541982, 33607369.14736999, 34509284.16610817
  34977977.00755431, 35216925.42095353, 35337571.86303162, 35398190.90625889
  35428574.73176835]
 [ 9702836.85189835, 13996061.58613707, 19224107.10330123, 24659123.88961204
  29047858.02041359, 31935351.61969705, 33607369.13082578, 34509284.14912088
  34977976.99033677, 35216925.40361861, 35337571.84563743, 35398190.88883494
  35428574.71432947]
 [ 9702836.84231433, 13996061.57232812, 19224107.08434998, 24659123.86531735
  29047857.99180546, 31935351.5882515,  33607369.09773736, 34509284.11514632
  34977976.9559017,  35216925.36894879, 35337571.81084908, 35398190.85398702
  35428574.67945171]
 [ 9702836.82314628 13996061.54471024 19224107.0464475  24659123.81672797
  29047857.93458923 31935351.52536041 33607369.03156052 34509284.04719717
  34977976.88703158 35216925.29960914 35337571.74127236 35398190.78429119
  35428574.60969618]
 [ 9702836.78481019 13996061.48947447 19224106.97064253 24659123.71954921
  29047857.82015678 31935351.39957821 33607368.89920683 34509283.91129888
  34977976.74929134 35216925.16092983 35337571.60211892 35398190.64489953
  35428574.47018512]
 [ 9702836.70813801 13996061.37900294 19224106.81903259 24659123.52519169
  29047857.59129186 31935351.14801385 33607368.63449948 34509283.63950231
  34977976.47381084 35216924.88357124 35337571.32381207 35398190.36611621
  35428574.19116299]
 [ 9702836.55479365 13996061.15805987 19224106.51581271 24659123.13647664
  29047857.13356202 31935350.64488509 33607368.10508474 34509283.09590916
  34977975.92284986 35216924.32885404 35337570.76719835 35398189.80854958
  35428573.63311874]
 [ 9702836.24810495 13996060.71617376 19224105.90937298 24659122.35904659
  29047856.2181024  31935349.63862765 33607367.04625536 34509282.00872292
  34977974.82092796 35216923.2194197  35337569.65397096 35398188.69341637
  35428572.51703028]
 [ 9702835.63472758 13996059.8324016  19224104.6964936  24659120.80418658
  29047854.38718329 31935347.62611289 33607364.92859671 34509279.83435059
  34977972.61708428 35216921.00055116 35337567.42751633 35398186.46315011
  35428570.28485353]
 [ 9702834.40797302 13996058.06485752 19224102.2707352  24659117.694467
  29047850.72534556 31935343.60108394 33607360.69328001 34509275.48560654
  34977968.20939755 35216916.56281471 35337562.97460769 35398182.00261821
  35428565.82050066]
 [ 9702831.95446459 13996054.52977038 19224097.41921975 24659111.47502962
  29047843.40167218 31935335.55102832 33607352.22264902 34509266.78812094
  34977959.39402659 35216907.68734434 35337554.06879296 35398173.08155695
  35428556.89179744]
 [ 9702827.04745052 13996047.45960009 19224087.71619435 24659099.03616189
  29047828.75433372 31935319.45092618 33607335.28139661 34509249.39315952
  34977941.76329467 35216889.93641363 35337536.25717357 35398155.23944449
  35428539.0344011 ]
 [ 9702817.23343348 13996033.31927552 19224068.31016551 24659074.15845459
  29047799.45968995 31935287.25075836 33607301.39893014 34509214.60327606
  34977906.50187068 35216854.43459237 35337500.63397507 35398119.55525994
  35428503.31964883]
 [ 9702797.60544388 13996005.03869042 19224029.49819569 24659024.40315258
  29047740.87053498 31935222.85056842 33607233.63415052 34509145.02366658
  34977835.97918228 35216783.43111052 35337429.38773929 35398048.18705235
  35428431.89030591]
 [ 9702758.3496425  13995948.47777635 19223951.87460747 24658924.89299898
  29047623.69275536 31935094.05077142 33607098.10520457 34509005.86507733
  34977694.93444374 35216641.4247894  35337286.89591251 35397905.45128305
  35428289.03226652]]

In [ ]:
def find_gamma_sigma_pair_with_lowest_MSE_using_gaussian_KRR(ds):
    _5_X_train, _5_y_train, _5_X_valid, _5_y_valid = generate_5_folds_from_dataset(ds)
    gammas, sigmas = generate_gammas_and_sigmas()
    mean_of_sqrd_errors_per_gs_pair_for_one_x_val = np.zeros((len(gammas), len(sigmas)))
    MSEs_for_each_gs_pair_for_all_5folds = []

    for X_train, y_train, X_val, y_val in zip(_5_X_train, _5_y_train, _5_X_valid, _5_y_valid):
        for g, gamma in enumerate(gammas):
            for s, sigma in enumerate(sigmas):
                a_stars_for_this_gs_pair_and_fold = solve_dual_optimisation(X_train=X_train, gamma=gamma,
                                                                            sigma=sigma, y_train=y_train)
                sqrd_errors = []
                for i, (x_val_row, y_val_row) in enumerate(zip(X_val, y_val)):
                    y_test_pred = evaluation_of_regression(a_stars=a_stars_for_this_gs_pair_and_fold,
                                                           X_train=X_train, X_val_row=x_val_row, sigma=sigma)

                    sqrd_errors.append(np.square(y_test_pred - y_val_row))

                assert len(sqrd_errors) == len(y_val), 'sqrd_errors list is not the expected length'
                mean_of_sqrd_errors_per_gs_pair_for_one_x_val[g][s] = np.mean(sqrd_errors)

        # HERE IS WHERE I HAVE SOME CONCERN THAT I SHOULD BE MAKING A COPY OR DEEPCOPY OF THE ARRAY, AS I MAY JUST END UP WITH 5 IDENTICAL VALUES IN THE 5 FOLDS
        MSEs_for_each_gs_pair_for_all_5folds.append(mean_of_sqrd_errors_per_gs_pair_for_one_x_val)
        mean_of_sqrd_errors_per_gs_pair_for_one_x_val = np.zeros((len(gammas), len(sigmas)))

    fold1 = MSEs_for_each_gs_pair_for_all_5folds[0]
    fold2 = MSEs_for_each_gs_pair_for_all_5folds[1]
    fold3 = MSEs_for_each_gs_pair_for_all_5folds[2]
    fold4 = MSEs_for_each_gs_pair_for_all_5folds[3]
    fold5 = MSEs_for_each_gs_pair_for_all_5folds[4]

    assert len(MSEs_for_each_gs_pair_for_all_5folds) == 5
    mean_of_5folds = (fold1 + fold2 + fold3 + fold4 + fold5) / 5
    index_of_lowest_mse = np.argmin(mean_of_5folds)
    g_of_gs_pair_of_lowest_mse, s_of_gs_pair_of_lowest_mse = np.unravel_index(index_of_lowest_mse, mean_of_5folds.shape)
    return mean_of_5folds, g_of_gs_pair_of_lowest_mse, gammas[g_of_gs_pair_of_lowest_mse], \
        s_of_gs_pair_of_lowest_mse, sigmas[s_of_gs_pair_of_lowest_mse]

In [98]:
def generate_gammas_and_sigmas() -> tuple:
    gammas = [2 ** pow for pow in list(range(-40, -25))]
    sigmas = []
    for pow in list(range(7, 14)):
        sigmas.append(2 ** pow)
        sigmas.append(2 ** (pow + 0.5))
    sigmas = sigmas[:-1]
    return gammas, sigmas

gammas, sigmas = generate_gammas_and_sigmas()
print(gammas)
print(len(gammas))
print(sigmas)
print(len(sigmas))

[9.094947017729282e-13, 1.8189894035458565e-12, 3.637978807091713e-12, 7.275957614183426e-12, 1.4551915228366852e-11, 2.9103830456733704e-11, 5.820766091346741e-11, 1.1641532182693481e-10, 2.3283064365386963e-10, 4.656612873077393e-10, 9.313225746154785e-10, 1.862645149230957e-09, 3.725290298461914e-09, 7.450580596923828e-09, 1.4901161193847656e-08]
15
[128, 181.01933598375618, 256, 362.03867196751236, 512, 724.0773439350247, 1024, 1448.1546878700494, 2048, 2896.309375740099, 4096, 5792.618751480198, 8192]
13


In [100]:
2**-40

9.094947017729282e-13

In [101]:
2**-26

1.4901161193847656e-08